# Setup

In [1]:
import pandas as pd
import numpy as np

import os
import re
from tqdm import tqdm

In [2]:
!pip install pyicu

In [3]:
import icu

# Split into 3 samples

In [4]:
def split_into_samples(input_file, set_name):
    df = pd.read_csv(input_file, sep='\t', encoding='utf-8-sig')
    samples = np.array_split(df, 3)
    
    for i, sample in enumerate(samples, 1):
        sample.to_csv(f'/kaggle/working/{set_name}_sample_{i}.tsv', sep='\t', encoding='utf-8-sig', index=False)
        print(f'Saved {set_name}_sample_{i}.tsv')

# Lọc tags

In [5]:
def filter_tags(df):
    content_lower = df['content'].replace('_', ' ').lower()
    tags_lower = [tag.lower() for tag in df['tags'].split(',')]
    filtered_tags_lower = [tag for tag in tags_lower if tag in content_lower]

    original_tags = df['tags'].split(',')
    tag_map = {tag.lower(): tag for tag in original_tags}

    filtered_tags = [tag_map[tag] for tag in filtered_tags_lower]

    return ','.join(filtered_tags)

In [6]:
def get_df_after_filter_tag(df):
    df.columns = ['content', 'tags']
    df['content'] = df['content'].fillna('')
    df['tags'] = df['tags'].fillna('')
    
    df['tags'] = df.apply(filter_tags, axis=1)
    return df

# Xử lý dữ liệu

## Lọc nhiễu lần 1

In [7]:
patterns = {
    'date': r'\b(?:0[1-9]|[12][0-9]|3[01])/(?:0[1-9]|1[0-2])/\d{4}\b|\b(?:0[1-9]|1[0-2])/(?:0[1-9]|[12][0-9]|3[01])/\d{4}\b',
    'long_numbers': r'\b\d{6,}(?:[.,]\d{3})*\b(?![A-Za-z])',
    'ip': r'\b(?:\d{1,3}\.){3}\d{1,3}\b|\b(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}\b',
    'percentage': r'\d+%',
    'time': r'\b\d{1,2}h(?:\d{2})?\b|\b\d{1,2}:\d{2}\b|\b\d{1,2}\+\d{1,2}\b',
    'comments': r'(Ảnh|Nguồn)\s*:\s*(?:@\s*)?\w+',
    'mail' : r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b',
    'url' : r'https?://(?:www\.)?\S+|www\.\S+|\b\S+\.(?:com|org|net|gov|edu|vn)\b'
}

In [8]:
def remove_patterns(text, patterns):
    for pattern in patterns.values():
        text = re.sub(pattern, '', text)
    return text

def filter_noise_first(df, set_name):
    content_list = df['content'].tolist()
    cleaned_content_list = df['content'].apply(lambda x: remove_patterns(x, patterns))
    
    if set_name == 'train':
        tags_list = df['tags'].tolist()
        cleaned_df = pd.DataFrame({'content': content_list, 'tags': tags_list})
    else:
        cleaned_df = pd.DataFrame({'content': content_list})

    return cleaned_df

## Lọc nhiễu lần 2 (Trừ '-' và '/')

In [9]:
def remove_special_char(text):
    # Loại bỏ khoảng trắng trước các dấu
    text = re.sub(r'\s+([!#$%&’()*+\\<=>?@\[\]^`{|}~\“\”\":.,;])', r'\1', text)

    # Xoá ký tự đặc biệt, ngoại trừ '-' và '/'
    special_char = "!#$%&’()*+<=>?@[]^`{|}~“”\":±'"
    translation_table = str.maketrans(special_char, " " * len(special_char))
    text = text.translate(translation_table)

    # Loại bỏ ký tự '/' và '-' nếu không có ký tự liền trước và liền sau
    text = re.sub(r'(?<!\w)[/-](?!\w)', '', text)

    # Loại bỏ khoảng trắng dư thừa do ký tự bị cách
    text = re.sub(r'\s{2,}', ' ', text)

    # Loại bỏ nhiều dấu chấm
    text = re.sub(r'\.', '', text)
    text = re.sub(r'…', '', text)

    # Xóa khoảng trắng thừa ở đầu và cuối câu
    return text.strip()

## Xử lý thành ngữ

### Tiền xử lý thành ngữ

In [10]:
def clean_text(text):
    text = text.lower()
    
    text = re.sub(r'[^\w\s]+$', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[\[\]]', '', text)
    
    text = re.sub(r'[\u4e00-\u9fff]+', '', text)  # loại bỏ tiếng Trung
    
    return text

def create_idiom_dict(tntn_web_num):
    df = pd.read_csv(f'/kaggle/input/idiom-org/tntn_{tntn_web_num}.csv')
    
    df = df.map(lambda x: clean_text(x) if isinstance(x, str) else x)
    
    df = pd.concat([df, df], ignore_index=True)
    
    df = df.drop_duplicates()
    
    return df

In [11]:
# tạo từ điển

idiom_df_all = pd.DataFrame()

for tntn_web_num in range(1, 5):
  idiom_df_all = create_idiom_dict(tntn_web_num)

# thêm tay một số trường hợp đặc biệt
new_tntn = ["lụa đẹp vì người", "không cánh mà bay", "không ít thì nhiều"]
idiom_df_new = pd.DataFrame(new_tntn, columns=[idiom_df_all.columns[0]])
idiom_df_all = pd.concat([idiom_df_all, idiom_df_new], ignore_index=True)

# sort theo tiếng Việt
collator = icu.Collator.createInstance(icu.Locale('vi', 'VN'))

df_sorted = idiom_df_all.sort_values(by=idiom_df_all.columns[0], key=lambda x: x.apply(lambda name: collator.getSortKey(name)))
df_sorted.reset_index(drop=True, inplace=True)

# set thanh ngu!!
idiom_set = set(df_sorted['Thành ngữ/Tục ngữ'].str.lower())

### Tách thành ngữ trong data theo từ điển


In [12]:
def tokenizer(text):
    modified_text = text
    
    for idiom in idiom_set:
        pattern = r'\b' + re.escape(idiom) + r'\b'
        if re.search(pattern, modified_text.lower()):
            modified_text = re.sub(pattern, lambda m: m.group(0).replace(" ", "_"), modified_text, flags=re.IGNORECASE)

    return modified_text

def process_paragraph(paragraph):
    sents = paragraph.split('. ')
    cleaned_sents = [tokenizer(sent) for sent in sents]
    return '. '.join(cleaned_sents)

def process_paragraphs(content_list):
    return [process_paragraph(p) for p in tqdm(content_list, desc="Processing paragraphs")]

def idiom_process(df, set_name):
    content_list = df['content'].tolist()
    idiom_content_list = process_paragraphs(content_list)

    if set_name == 'train':
        tags_list = df['tags'].tolist()
        cleaned_df = pd.DataFrame({'content': idiom_content_list, 'tags': tags_list})
    else:
        cleaned_df = pd.DataFrame({'content': idiom_content_list})
    
    return cleaned_df

# Preprocess

In [13]:
def preprocess(df, sample_num, set_name):
    output_path = f'/kaggle/working/{set_name}_preprocessing1_{sample_num}.csv'

    # Xác định giá trị bắt đầu cho `idx`
    if sample_num == 1:
        if set_name == 'test':
            start_idx = 0
        else:
            test_file_path = '/kaggle/working/test_preprocessing1_3.csv'

            test_df = pd.read_csv(test_file_path)
            start_idx = test_df['idx'].max() + 1
    else:
        prev_output_path = f'/kaggle/working/{set_name}_preprocessing1_{sample_num - 1}.csv'
        prev_df = pd.read_csv(prev_output_path)
        if 'idx' in prev_df.columns:
            start_idx = prev_df['idx'].max() + 1

    if set_name == 'train':
        df = get_df_after_filter_tag(df)
    
    df = filter_noise_first(df, set_name)
    df['content'] = df['content'].apply(remove_special_char)
    df = idiom_process(df, set_name)
    
    df['idx'] = range(start_idx, start_idx + len(df))

    cols = ['idx'] + [col for col in df.columns if col != 'idx']
    df = df[cols]

    df.to_csv(output_path, index=False, encoding='utf-8-sig')

# Chạy

In [14]:
set_names = ["test", "train"]

for set_name in set_names:
    print(f"Processing for set_name: {set_name}")
    if set_name == "test":
        split_into_samples('/kaggle/input/org-test-data/articles_testing.tsv', set_name)
    else:
        split_into_samples('/kaggle/input/original-articles/articles_training.tsv', set_name)

    for sample_num in range(1, 4): 
        df = pd.read_csv(f'/kaggle/working/{set_name}_sample_{sample_num}.tsv', sep='\t', encoding='utf-8-sig')
        preprocess(df, sample_num, set_name)

Processing for set_name: test


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Saved test_sample_1.tsv
Saved test_sample_2.tsv
Saved test_sample_3.tsv


Processing paragraphs: 100%|██████████| 14634/14634 [01:44<00:00, 140.64it/s]


Processing for set_name: train


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Saved train_sample_1.tsv
Saved train_sample_2.tsv
Saved train_sample_3.tsv


Processing paragraphs: 100%|██████████| 33283/33283 [05:27<00:00, 101.69it/s]
